In [52]:
import numpy as np
import matplotlib.pyplot as plt

In [66]:
#case of a mean criterion with penalty on inventory
def U(x):
    return x

def g(y_):
    return y_**2

def g2(y_):
    return abs(y_)

#devrait fonctionner

In [54]:
def simul_poisson_proc(intensity_,t):
    return 0

In [55]:
#test ipynb cell
#Should return results similar to that of the article studied
#might take around 30-50min to be executed

rho = np.array([[0,0.410,0.220,0.160,0.142,0.065],
              [0.201,0,0.435,0.192,0.103,0.067],
              [0.113,0.221,0,0.4582,0.147,0.059],
              [0.070,0.085,0.275,0,0.465,0.102],
              [0.068,0.049,0.073,0.363,0,0.446],
              [0.077,0.057,0.059,0.112,0.692,0]])

#MM_strategy(U,0.8,g,100,100,rho)

In [76]:
class Market_maker:
    def __init__(self, U_,gamma_,g_,l_max,e_max,rho_, type_rebate = "both") -> None:
        #Market parameters
        self.delta_s = 0.005
        self.rho_rebate = 0.0008
        self.epsilon = 0.0012
        self.epsilon0 = 10**(-6)
        self.lambd_t = 1 #s^(-1)
        #Optimization parameters (parameter of the function)
        #Discretization/localization parameters
        self.T = 300
        self.y_min = -1000
        self.y_max = 1000
        self.n = 100
        self.m = 6
        #Backtest parameters
        self.delta_t = 0.3
        self.l_0 = 100 #not necessary
        self.x_0 = 0
        self.y_0 = 0
        self.p_0 = 45
        #
        #Grid mesh
        self.delta_inventory = 5
        self.len_time_grid = int(self.T//self.delta_t + 1)
        self.len_spread_grid = 6 # = m in the article
        self.len_inventory_grid = (self.y_max - self.y_min)//self.delta_inventory +1
        
        self.g = g_
        self.gamma = gamma_/g_(self.y_max - self.y_min) #the denominator for gamma has been added
        #the reason is that the results with g(x) = x**2 and gamma aren't good
        self.lmax = l_max
        self.emax = e_max
        self.R = self.lambd_t*rho_
        for k in range(len(self.R)):
            #self.R[k,k] = 1 - (sum(self.R[k,:]) - self.R[k,k])
            self.R[k,k] = - (sum(self.R[k,:]) - self.R[k,k])
            #self.R[k,k] = np.exp(-lambd_t*delta_t)
            #self.R[k,k] = 1/delta_t - lambd_t
        # print("R =",self.R)
        
        self.type_rebate = type_rebate
        #Symmetrical values of lambda in s^(-1)
        self.B__intensity = [0.063,0.049,0.041,0.0385,0.044,0.0584]
        self.B_mp_intensity = [0.1625,0.1060,0.088,0.0876,0.0970,0.1228]
        
        #True values of lambda in s^(-1)
        self.Bb_intensity = [0.0718,0.052,0.0419,0.0409,0.0452,0.0614]
        self.Bbp_intensity = [0.1763,0.1144,0.0915,0.0896,0.0930,0.1255]
        self.Ba_intensity = [0.0539,0.0465,0.0401,0.0360,0.0435,0.0554]
        self.Bam_intensity = [0.1485,0.0979,0.0846,0.0856,0.1009,0.1202]
        
        #Initialising strategy arrays
        self.array_phi = np.empty(shape=(self.len_time_grid, self.len_inventory_grid, self.len_spread_grid))
        self.array_v = np.empty(shape=(self.len_time_grid, self.len_inventory_grid, self.len_spread_grid))
        self.tab_strategy = [ [ [ [] for ks in range(self.len_spread_grid)] for ky in range(self.len_inventory_grid)] for kt in range(self.len_time_grid)]
        self.expected_profits = np.empty(shape=(self.len_time_grid, self.len_inventory_grid, self.len_spread_grid))
        self.constant_state = np.empty(shape=(self.len_time_grid, self.len_inventory_grid, self.len_spread_grid))
        self.boundary_conditions()
    
    def get_phi(self):
        return self.array_phi
    
    def get_v(self):
        return self.array_v
    
    def get_strategy(self):
        return self.tab_strategy
    
    def get_expected(self):
        return self.expected_profits
    
    def get_constant(self):
        return self.constant_state
    
    def to_inventory_delta(self,e):
        return e//self.delta_inventory
    
    def to_inventory_index(self,y):
        return self.to_inventory_delta(y - self.y_min)
    
    def to_inventory_value(self,e):
        return e*self.delta_inventory
    
    def to_inventory_range(self, y):
        return self.to_inventory_value(y) + self.y_min
    
    def to_time_index(self,t):
        return t//self.delta_t
    
    def to_time_range(self,t):
        return t*self.delta_t
    
    def to_spread_index(self, i):
        return i-1
    
    def to_spread_value(self, i):
        return i+1

    def lambd_sym(self, ab,q,i_):
        if ab =="b":
            if q=="Bb":
                return self.B__intensity[i_]
            if q=="Bbp":
                return self.B_mp_intensity[i_]
        if ab =="a":
            if q=="Ba":
                return self.B__intensity[i_]
            if q=="Bam":
                return self.B_mp_intensity[i_]
        else:
            print("error lambd")
            return

    def lambd_true(self, ab,q,i_): #we use the actual estimated values from the article
        if ab =="b":
            if q=="Bb":
                return self.Bb_intensity[i_]
            if q=="Bbp":
                return self.Bbp_intensity[i_]
        if ab =="a":
            if q=="Ba":
                return self.Ba_intensity[i_]
            if q=="Bam":
                return self.Bam_intensity[i_]
        else:
            print("error lambd")
            return

    def pi_price(self, ab, q, i_, type_rebate = "both"):
        i_ = i_ + 1 #We take the value associated to the index so that i_*delta_s_ = s
        #Sould we take the price into account ?? I don't think so
        if type_rebate=="proportional":
            if ab == "b":
                return (-i_*self.delta_s/2 + self.delta_s*(q=="Bbp"))*(1 - self.rho_rebate)
            elif ab == "a":
                return (i_*self.delta_s/2 - self.delta_s*(q=="Bam"))*(1 + self.rho_rebate)
            else:
                print("error pi_price : ab")
                return
        elif type_rebate=="per share":
            if ab == "b":
                return -i_*self.delta_s/2 + self.delta_s*(q=="Bbp") - self.epsilon
            elif ab == "a":
                return i_*self.delta_s/2 - self.delta_s*(q=="Bam") + self.epsilon
            else:
                print("error pi_price : ab")
                return
        elif type_rebate=="both":
            if ab == "b":
                return (-i_*self.delta_s/2 + self.delta_s*(q=="Bbp"))*(1 - self.rho_rebate) - self.epsilon
            elif ab == "a":
                return (i_*self.delta_s/2 - self.delta_s*(q=="Bam"))*(1 + self.rho_rebate) + self.epsilon
            else:
                print("error pi_price : ab")
                return
        else:
            print("error pi_price : type_rebate")
            return

    def cost0(self, _e_, i_):
        i_ = self.to_spread_value(i_) #We take the value associated to the index so that i_*delta_s_ = s
        #fixed_fee = epsilon_0 #Actually useless but it's better for comprehension
        return abs(self.to_inventory_value(_e_))*i_*self.delta_s/2 + self.epsilon0

    def cost(self, _e_, i_, type_fees = "both"):
        if type_fees in ["base",0,"0"]:
            return self.cost0(_e_, i_)
        i_ = self.to_spread_value(i_) #We take the value associated to the index so that i_*delta_s_ = s
        _e_ = self.to_inventory_value(_e_) #We take the actual number of asset associated to the number of the index
        if type_fees=="proportional": #Sould we take the price into account ?? I don't think so
            return (abs(_e_) + self.rho_rebate*_e_)*i_*self.delta_s/2 + self.epsilon0 #+ fixed_fee*abs(_e_)*price #??
        elif type_fees=="per share":
            return abs(_e_)*(i_*self.delta_s/2 + self.epsilon) + self.epsilon0
        elif type_fees=="both":
            return (abs(_e_) + self.rho_rebate*_e_)*(i_*self.delta_s/2 + self.epsilon) + self.epsilon0
        else:
            print("error pi_price : type_fees")
            return

    def sup_HJB(self, phi_t,ab,i_,y_):
        if ab=="a":
            output_sup = [0,"Ba",0] #base case when the optimal l is 0
            for l in range(self.to_inventory_delta(self.lmax)):
                if y_ - l >=0:
                    if i_ == 0:
                        tmp = self.lambd_sym(ab,"Ba",i_)*( phi_t[y_ - l,i_] - phi_t[y_, i_] + self.pi_price(ab, "Ba", i_, self.type_rebate)*self.to_inventory_value(l) )
                        if output_sup[0] < tmp:
                            output_sup = [tmp,"Ba",l]
                    else:
                        for q_ in ["Ba","Bam"]:
                            tmp = self.lambd_sym(ab,q_,i_)*( phi_t[y_ - l,i_] - phi_t[y_, i_] + self.pi_price(ab, q_, i_, self.type_rebate)*self.to_inventory_value(l) )
                            if output_sup[0] < tmp:
                                output_sup = [tmp,q_,l]
        elif ab=="b":
            output_sup = [0,"Bb",0] #base case when the optimal l is 0
            for l in range(self.to_inventory_delta(self.lmax)):
                if y_ + l <self.len_inventory_grid:
                    if i_ == 0:
                        tmp = self.lambd_sym(ab,"Bb",i_)*( phi_t[y_ + l,i_] - phi_t[y_, i_] - self.pi_price(ab, "Bb", i_, self.type_rebate)*self.to_inventory_value(l) )
                        if output_sup[0] < tmp:
                            output_sup = [tmp,"Bb",l]
                    else:
                        for q_ in ["Bb","Bbp"]:
                            tmp = self.lambd_sym(ab,q_,i_)*( phi_t[y_ + l,i_] - phi_t[y_, i_] - self.pi_price(ab, q_, i_, self.type_rebate)*self.to_inventory_value(l) )
                            if output_sup[0] < tmp:
                                output_sup = [tmp,q_,l]
        return output_sup

    def sum_R_phi(self, phi_ty,i_):
        sum_output = 0
        for j in range(len(self.R)):
            sum_output = sum_output + self.R[i_,j]*(phi_ty[j] - phi_ty[i_])
            #print("+= ",self.R[i_,j], "*", (phi_ty[j] - phi_ty[i_]), "=", self.R[i_,j]*(phi_ty[j] - phi_ty[i_]))
        return sum_output

    def sup_MO(self, phi_t_i,y_, i_):
        output_sup = [phi_t_i[y_],0]
        _e_ = self.to_inventory_delta(self.emax)
        for e in range(-_e_,_e_ +1):
            if (y_ + e >= 0) and (y_ + e < self.len_inventory_grid):
                tmp = phi_t_i[y_ + e] - self.cost(e, i_)
                if output_sup[0] < tmp:
                    output_sup = [tmp,e]
        return output_sup
    
    def boundary_conditions(self):
        #We set up the boundary conditions
        #We take phi_i(t,y_max) = phi_i(t,y_min) = -gamma*g(y_max+ymin) as boundary condition because we have no interest in having a too big inventory
        #we could extend it to phi_i(t,-_e*(T-t)/delta_t) = phi_i(t,_e*(T-t)/delta_t) = -(_e*(T-t)/delta_t)*i*delta/2 -epsilon*(T-t)/delta_t
        for y in range(self.len_inventory_grid):
            for i in range(self.len_spread_grid):
                #boundary_penalty = self.gamma*self.g( max(self.to_inventory_range(y) - self.emax,0) + min(self.to_inventory_range(y) + self.emax,0) )
                self.array_phi[-1,y,i] = -self.cost0( self.to_inventory_delta( self.to_inventory_range(y) ) , i) #- boundary_penalty*100
                self.tab_strategy[-1][y][i] = [self.array_phi[-1,y,i],"Bb","Ba",0,0,0]
                self.expected_profits[-1,y,i] = -self.cost0( self.to_inventory_delta( self.to_inventory_range(y) ) , i)
                self.constant_state[-1,y,i] = -self.cost0( self.to_inventory_delta( self.to_inventory_range(y) ) , i)
        #My other boundary conditions I could add
    #    for t in range(len_time_grid):
    #        for i in range(len_spread_grid):
    #            array_phi[t,len_inventory_grid-1,i] = -abs(y_max)*(i+1)*delta_s/2 -epsilon0  -gamma_*g_(y_max)*(T-t)
    #            array_phi[t,0,i] = -abs(y_min)*(i+1)*delta_s/2 -epsilon0  -gamma_*g_(y_min)*(T-t)

    def MM_strategy(self):
        print("time:", self.len_time_grid,", spread:", self.len_spread_grid,", inventory:", self.len_inventory_grid)
        #
        #We solve the system
        for t in range(self.len_time_grid-1,-1,-1):
            # if t%10==0: 
            #     print("t = ",t)
            for i in range(self.len_spread_grid):
                for y in range(self.len_inventory_grid):
                    q_l_a = self.sup_HJB(self.array_phi[t,:,:],"a",i,y)
                    q_l_b = self.sup_HJB(self.array_phi[t,:,:],"b",i,y)
                    tmp = self.sum_R_phi(self.array_phi[t,y,:],i)
                    self.array_phi[t -1,y,i] = self.array_phi[t,y,i] + self.delta_t*(tmp + q_l_b[0] + q_l_a[0] - self.gamma*self.g(self.to_inventory_range(y)))
                    self.tab_strategy[t-1][y][i] = [self.array_phi[t-1,y,i],q_l_b[1],q_l_a[1],q_l_b[2],q_l_a[2]]
                    # self.expected_profits[t -1,y,i] = self.expected_profits[t,y,i] + self.delta_t*(tmp + q_l_b[0] + q_l_a[0]) #Changer par valeur de expected
                    #A tester :
                    self.expected_profits[t-1,y,i] = self.expected_profits[t,y,i]
                    self.expected_profits[t-1,y,i] += self.delta_t*self.sum_R_phi(self.expected_profits[t,y,:],i) #equivalent of tmp
                    self.expected_profits[t-1,y,i] += self.delta_t*( self.lambd_sym("b",q_l_b[1],i)*( self.expected_profits[t,y + q_l_b[2],i] - self.expected_profits[t,y, i] - self.pi_price("b", q_l_b[1], i)*self.to_inventory_value(q_l_b[2]) ) ) #equivalent to q_l_b[0]
                    self.expected_profits[t-1,y,i] += self.delta_t*( self.lambd_sym("a",q_l_a[1],i)*( self.expected_profits[t,y - q_l_a[2],i] - self.expected_profits[t,y, i] + self.pi_price("a", q_l_a[1], i)*self.to_inventory_value(q_l_a[2]) ) ) #equivalent to q_l_a[0]
                    #Benchmark strategy
                    self.constant_state[t-1,y,i] = self.constant_state[t,y,i]
                    self.constant_state[t-1,y,i] += self.delta_t*self.sum_R_phi(self.constant_state[t,y,:],i) #equivalent of tmp
                    self.constant_state[t-1,y,i] += self.delta_t*( self.lambd_sym("b","Bb",i)*( self.constant_state[t,min(y + self.to_inventory_delta(self.lmax), self.to_inventory_index(self.y_max)),i] - self.constant_state[t,y, i] - self.pi_price("b", "Bb", i)*self.to_inventory_value(min(y + self.to_inventory_delta(self.lmax), self.to_inventory_index(self.y_max)) - y) ) ) #equivalent to q_l_b[0]
                    self.constant_state[t-1,y,i] += self.delta_t*( self.lambd_sym("a","Ba",i)*( self.constant_state[t,max(y - self.to_inventory_delta(self.lmax), self.to_inventory_index(self.y_min)),i] - self.constant_state[t,y, i] + self.pi_price("a", "Ba", i)*self.to_inventory_value(y - max(y - self.to_inventory_delta(self.lmax), self.to_inventory_index(self.y_min))) ) ) #equivalent to q_l_a[0]
                    # print("sum R phi =", tmp)
                    # print("sup ask =", q_l_a)
                    # print("sup bid =", q_l_b)
            #
            #We apply  market order mechanism
            for i in range(self.len_spread_grid):
                for y in range(self.len_inventory_grid):
                    mo_e = self.sup_MO(self.array_phi[t-1,:,i],y,i)
                    self.array_phi[t-1,y,i] = mo_e[0]
                    self.tab_strategy[t-1][y][i].append(mo_e[1])
                    #tab_strategy[t-1][y][i][0] = array_phi[t-1,y,i]
                    self.expected_profits[t-1,y,i] = self.expected_profits[t-1,y + mo_e[1],i] - self.cost(mo_e[1], i)
        #
        for y in range(self.len_inventory_grid):
            self.array_v[:,y,:] = self.x_0 + self.to_inventory_range(y)*self.p_0 + self.array_phi[:,y,:]

        print("Données en sortie : (tableau d'estimation des gains moyens de la stratégie , tableau des instructions pour suivre la stratégie)")
        print("La structure des tableaux de données en sortie de cette fonction est [indice de temps, indice de quantité d'actifs en inventaire, indice de valeur de spread (en nombre de tick)]")
        print("Chaque valeur du tableau d'instructions est une liste sous la forme :")
        print("[valeur de phi avant market order, prix auquel on place le bid Limit Order, prix auquel on place le ask LO, quantité d'actifs placé au bid LO, quantité d'actifs placé au ask LO, quantité d'actifs à échanger via Market Order]")
        print("Les gains estimé à t = 0, pour un inventaire vide sont de ", self.expected_profits[0, self.to_inventory_index(0)])

In [57]:
test = Market_maker(U,0,g,100,100,rho)
for i in range(6):
    print("i=", i)
    print("diff cost",test.cost(1, i) - test.cost(-1,i))
    print("diff cost0",test.cost0(1, i) - test.cost0(-1,i))
    print("cost0",test.cost0(1, i))
    print("diff pi_price",test.pi_price("a", "Ba", i) + test.pi_price("b", "Bb", i))
    print("pi_price ask:", test.pi_price("a", "Ba", i), ", pi_price bid:", test.pi_price("b","Bb", i))

i= 0
diff cost 2.9599999999994214e-05
diff cost0 0.0
cost0 0.012501
diff pi_price 3.9999999999996635e-06
pi_price ask: 0.0037019999999999996 , pi_price bid: -0.003698
i= 1
diff cost 4.95999999999934e-05
diff cost0 0.0
cost0 0.025001000000000002
diff pi_price 8.000000000000194e-06
pi_price ask: 0.0062039999999999994 , pi_price bid: -0.006195999999999999
i= 2
diff cost 6.959999999998912e-05
diff cost0 0.0
cost0 0.037501
diff pi_price 1.1999999999998123e-05
pi_price ask: 0.008705999999999998 , pi_price bid: -0.008694
i= 3
diff cost 8.95999999999883e-05
diff cost0 0.0
cost0 0.050001000000000004
diff pi_price 1.600000000000039e-05
pi_price ask: 0.011208 , pi_price bid: -0.011191999999999999
i= 4
diff cost 0.00010959999999998749
diff cost0 0.0
cost0 0.062501
diff pi_price 1.9999999999999185e-05
pi_price ask: 0.01371 , pi_price bid: -0.01369
i= 5
diff cost 0.0001295999999999936
diff cost0 0.0
cost0 0.075001
diff pi_price 2.3999999999996247e-05
pi_price ask: 0.016211999999999997 , pi_price bid

In [77]:
test2 = Market_maker(U,0.8,g2,100,100,rho)
#test2 = Market_maker(U,6e-5,g,100,100,rho)
test2.MM_strategy()

time: 1001 , spread: 6 , inventory: 401
Données en sortie : (tableau d'estimation des gains moyens de la stratégie , tableau des instructions pour suivre la stratégie)
La structure des tableaux de données en sortie de cette fonction est [indice de temps, indice de quantité d'actifs en inventaire, indice de valeur de spread (en nombre de tick)]
Chaque valeur du tableau d'instructions est une liste sous la forme :
[valeur de phi avant market order, prix auquel on place le bid Limit Order, prix auquel on place le ask LO, quantité d'actifs placé au bid LO, quantité d'actifs placé au ask LO, quantité d'actifs à échanger via Market Order]
Les gains estimé à t = 0, pour un inventaire vide sont de  [32.22344003 32.2387648  32.26819326 32.32282825 32.41992321 32.50291069]


In [83]:
print(test2.get_strategy()[0][test2.to_inventory_index(0)])
print(test2.get_expected()[0][test2.to_inventory_index(0)])
print(test2.get_constant()[0][test2.to_inventory_index(0)])

[[23.154656075846535, 'Bb', 'Ba', 19, 19, 0], [23.175560845979184, 'Bb', 'Ba', 19, 19, 0], [23.205670164717947, 'Bb', 'Ba', 19, 19, 0], [23.252891808442637, 'Bb', 'Ba', 19, 19, 0], [23.32379715852037, 'Bbp', 'Bam', 19, 19, 0], [23.39462415136344, 'Bbp', 'Bam', 19, 19, 0]]
[32.22344003 32.2387648  32.26819326 32.32282825 32.41992321 32.50291069]
[26.73663982 26.75256716 26.77609531 26.81520214 26.87125253 26.92646709]


In [ ]:
test = Market_maker(U,0,g,100,100,rho)
print(test.to_inventory_index(test.y_max))
print(test.len_inventory_grid)
print(test.len_spread_grid)
print(test.len_time_grid)
test.MM_strategy()

400
401
6
1001
time: 1001 , spread: 6 , inventory: 401
Données en sortie : (tableau d'estimation des gains moyens de la stratégie , tableau des instructions pour suivre la stratégie)
La structure des tableaux de données en sortie de cette fonction est [indice de temps, indice de quantité d'actifs en inventaire, indice de valeur de spread (en nombre de tick)]
Chaque valeur du tableau d'instructions est une liste sous la forme :
[valeur de phi avant market order, prix auquel on place le bid Limit Order, prix auquel on place le ask LO, quantité d'actifs placé au bid LO, quantité d'actifs placé au ask LO, quantité d'actifs à échanger via Market Order]
Les gains estimé à t = 0, pour un inventaire vide sont de  [35.93552222 35.94962721 35.98108214 36.05169044 36.1446749  36.22941133]


In [ ]:
test.get_phi()[-1, test.to_inventory_index(0)]
test.get_strategy()[0][220]
test.get_constant()[0][200]

array([26.73663982, 26.75256716, 26.77609531, 26.81520214, 26.87125253,
       26.92646709])

In [ ]:
print(test.get_expected()[0][200])
print(test.get_strategy()[0][200])

[35.93552222 35.94962721 35.98108214 36.05169044 36.1446749  36.22941133]
[[35.93652080590245, 'Bb', 'Ba', 19, 19, 0], [35.95062579737753, 'Bb', 'Ba', 19, 19, 0], [35.98208072493236, 'Bb', 'Ba', 19, 19, 0], [36.05268902489353, 'Bbp', 'Bam', 19, 19, 0], [36.14567348157814, 'Bbp', 'Bam', 19, 19, 0], [36.23040990905327, 'Bbp', 'Bam', 19, 19, 0]]


Check les fonctions pi_price puis cost